In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import sys
sys.path.insert(0,"..")

# Import packages and setup base classes

In [15]:
import torch
import torch.optim as optim
from types import SimpleNamespace as Namespace

In [16]:
from utils.training_loop import TrainingLoop
from custom_datasets.messidor import Messidor
from utils.construct_model import LoadVICRegModel
from utils.training_loop import TrainingLoop


In [10]:
gpu = torch.cuda.current_device()
torch.cuda.set_device(gpu)
torch.backends.cudnn.benchmark = True

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [17]:
dataset = Messidor(
    batch_size=16,
    num_workers=4,
    gpu=None,
)

# namespace mock

In [26]:
args = Namespace(
    batch_size=16,
    num_workers=4,
    gpu=None,
    weight_decay=0.0001,
    epochs=2,
    weights="finetune",
    print_freq=100,
)


# Infra

## dataloaders

In [19]:
train_loader = dataset.get_dataloader(split="train")
val_loader = dataset.get_dataloader(split="valid")

length of dataset train: 1394
length of dataset valid: 350


## model

In [22]:
model = LoadVICRegModel(arch="resnet50")
model.load_pretrained_weights(pretrained_path="../VICReg_ImageNet/resnet50.pth")
model.modify_head(num_classes=dataset.num_classes)
model = model.produce_model()
# model.cuda(gpu)
param_groups = [dict(params=model[-1].parameters(), lr=0.03)]

Loading local VICReg ResNet50 arch Model


In [23]:
## optimizer
optimizer = optim.Adam(param_groups, 0, weight_decay=args.weight_decay)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, args.epochs)


# training loop


In [30]:
training_loop = TrainingLoop(
    model,
    optimizer,
    scheduler,
    train_loader,
    val_loader,
    dataset,
    args,
    gpu=gpu,
    stats_file=None
)

In [31]:
training_loop.train(start_epoch=0, num_epochs=args.epochs)

2025-01-21 05:33 [info     ] Beginning training            


RuntimeError: Invalid device, must be cuda device